# **Task**

An assistant program will take natural language text as input and call the suitable function with specified parameters or do the correct calculations. Select one of the following tasks.

### **TASK #1**
```python
sam_measure(name="sample1", exposure_time=10, angle=[0.1])
```

This function triggers the hardware to measure the sample for 10 seconds (0.5s or more is reasonable) at a sample tilt angle theta of 0.1 degree (up to 0.5 degree, can be a list e.g. [0.1, 0.2] for measuring at multiple thetas). Both the time and angle are required parameters, sample/material name is optional.

Example inputs:
*  "I want to measure this polymer sample for ten seconds at an incident angle of 0.1 degree"

*  "Collect data for this material. Let us do an exposure time of 10 seconds and incident angles at 0.1 and 0.2 degree".

*  "We want to look at this perovskite sample to understand its structure. We think 5 seconds of exposure should be sufficient. Theta of 0.2 would be good."



### **TASK #2**

Given that
```
d = 2π / q
q = 4π sin(θ) / λ
```

Example inputs:
*  "For structure with d spacing of 6, what is the corresponding q in the reciprocal/Fourier space?"

*  "When we have q peak of 1, what is the d spacing?"

*  "If q is 1, and the wavelength lambda λ is 1, what is the scattering angle θ in degree? What is the d spacing?"


### **TASK #3**
Assume that the data reduction step (i.e. data processing, data analysis protocol, result) is defined as summation over a 2D array (i e. image, scattering data, collected/acquired data, measurement). 

```python
def data_reduction(image, dim=0):
    curve_1d = sum(array_2d, dim)
    return curve_1d
```

Example inputs:
"Show us the result for the latest measurement"
"Run data analysis"
"Process the latest data"


NOTE:
1. All information can be given in one NL text paragraph.
2. (Optional) Not all information/parameter are given at once, follow up questions by the assistant program are needed.
3. Open source model (e.g. Llama2, BERT) preferred
4. Working code is preferred, or provide the thought process and methods/reference

# **Task Submission:** This Notebook contains the working code, implementation description of task 1 and task 2, and further thoughts to build a real-time system with improved performance and accuracy.

I have worked on `Task #1` and `Task #2`.

Due to the lack of training datasets and fine-tuned models for the specific task, it was really hard to build a robust and well-generalized model. 

I used examples given in tasks 1 and 2 as seed examples and generated around 50 synthesized samples. The dataset can be found [here](https://github.com/geo47/bnl_task/tree/master/TaskRecognition/dataset).

### Task 1:

For task 1, I implement a goal-oriented assistant, whose goal is to recognize user intent and extract required entities to call a function (ex. sample name, exposure time, incident angle) from natural text query.

The steps of the task implementation are briefly defined as follows:
1. [Annotate dataset](https://github.com/geo47/bnl_task/tree/master/TaskRecognition/dataset/raw_files) for intent & entities (used task examples)
2. Train a [T5-base](https://arxiv.org/pdf/1910.10683.pdf) model on annotated dataset for Feature extraction.

Features includes ```(intent:["task1", "task2"], entities:["sample_name", "exposure_time", "angle"])```

Given a user query:
- The program first recognizes the intent (Type of task).
- If the intent is `'task_1'`, it will then pass on to the assistant, who asks for the required entity to perform the hardware operation job.


### Task 2:

I fine-tuned a `Llama-2-7b-chat-hf` model on `14 data samples` created from given examples in task 2. Due to hardware resource limitations in my notebook, I used colab for Task 2 and used T4 GPU _*(free session)*_.

### Task 2 Colab URL
`https://colab.research.google.com/drive/1v_vbcmsYiUP_RzkHGFYUZqwU4TJavFy5`

The steps of the task implementation are briefly defined as follows:
1. [Annotate dataset](https://colab.research.google.com/drive/1v_vbcmsYiUP_RzkHGFYUZqwU4TJavFy5#scrollTo=sV0D2rjRbk4z) for in instruction based format required to train a llama model (used task examples)
2. Train a [Llama-2-7b-chat-hf](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) model on annotated dataset for Maths operation

```json
{
  "instruction": "Given eq 1: d=2π/q, and eq 2: q=4πsin(θ)/λ, For structure with d spacing of 6, what is the corresponding q in the reciprocal/Fourier space?",
  "input": "",
  "output": "Given a spacing d=6, the corresponding q can be calculated with eq 1: d=2π/q:\nsubstitute the value of d in eq 1: 6=2π/q\nNow, solve for q:\nq=2π/6\nSimplify:\nq=π/3\n\nSo, if a material exhibits a spacing between crystal lattice planes d of 6, this corresponds to a reciprocal/Fourier space scattering vector q of π/3."
}
```
Given a user query:
- The program first recognizes the intent (Type of task).
- If the intent is `'task_2'`, it will then pass on to the Llama assistant to perform the calculations.

### **Install Dependencies and restart Runtime**

In [ ]:
!pip install langchain transformers datasets nltk pytorch-lightning==1.5.9 pip torchtext==0.6.0 seqeval

In [ ]:
# Dependencies for training and using Llama2 model.
!pip uninstall -y tensorflow --quiet
!pip install ludwig ludwig[llm]

In [1]:
import os
import warnings

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### **Task 1**

Training a T5-base model for feature extraction.

The model will be able to recognize the task intent (**task_1** or **task_2**) based on the given query. It can further extract the entities required for task 1 (**sample_name**, **exposure_time**, **angle**). 

Based on the task intent the program will run the corresponding task.

For Task 1 deliverables, I provide the code and results of **Feature-extractor model** _*training*_, _*evaluation*_, and _*inference*_.

In [2]:
""" 
MODEL TRAINING:

You can check the training code from the file. For your reference, the hyper-params for fine-tuning model are given below:

model_name_or_path='t5-small',
tokenizer_name_or_path='t5-small',
max_seq_length=40,
learning_rate=3e-4,
weight_decay=0.0,
adam_epsilon=1e-8,
warmup_steps=0,
train_batch_size=1,
eval_batch_size=1,
num_train_epochs=60,
gradient_accumulation_steps=16,
n_gpu=1,
early_stop_callback=False,
fp_16=True,
opt_level='O1',
max_grad_norm=1,
seed=42,
"""

%cd TaskRecognition/model
!python train_ner.py
%cd ../../

/data/Muzamil/projects/my_project/bnl_task/TaskRecognition/model
[nltk_data] Downloading package punkt to /home/w02/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
50
can we measure this sample</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
task_no: task_1</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
the exposure time for this experiment is set at 8 seconds.</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
task_no: task_1; exposure_time: 8</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Using 16bit native Automatic Mixed Precision (AMP)
/home/w02/anaconda3/envs/venv_task/lib

In [12]:
# MODEL EVALUATION:

%cd TaskRecognition/model
!python evaluate_ner.py
%cd ../../

/data/Muzamil/projects/my_project/bnl_task/TaskRecognition/model
50
can we measure this sample</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
task_no: task_1</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
  0%|                                                    | 0/50 [00:00<?, ?it/s]/home/w02/anaconda3/envs/venv_task/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/data/Muzamil/projects/my_project/bnl_task/TaskRecognition/model/evaluate_ner.py:95: FutureWarning: load_metric is deprecated and will be removed in the next major version o

In [7]:
############
# PLEASE NOTE THAT:
# Model performance to extract entities on short command is good. However, for longer input commands, it lacks to extract required entities.
# The reason is the lack of training dataset. (SEE result1 vs result2)
####


from TaskRecognition.rec_model import ExtractNER

ex_feature = ExtractNER()

query = "The exposure time for this experiment is set at 8 seconds."
query2 = "We want to look at this perovskite to understand its structure. We think 5 seconds of exposure should be sufficient. Theta of 0.2 would be good."
query3 = "When the reciprocal space scattering vector q peaks at 1, what does this imply for the interplanar spacing d?"

result1 = ex_feature.predict_ner(query)
result2 = ex_feature.predict_ner(query2)
result3 = ex_feature.predict_ner(query3)

print(result1)
print("\n")
print(result2)
print("\n")
print(result3)

{'input': 'the exposure time for this experiment is set at 8 seconds.', 'ner': [{'exposure_time': '8'}, {'task_no': 'task_1'}]}


{'input': 'we want to look at this perovskite to understand its structure. we think 5 seconds of exposure should be sufficient. theta of 0.2 would be good.', 'ner': [{'sample_name': 'perovskite'}, {'task_no': 'task_1'}]}


{'input': 'when the reciprocal space scattering vector q peaks at 1, what does this imply for the interplanar spacing d?', 'ner': [{'task_no': 'task_2'}]}


### **Task 2**

Please refer to the link given below for the `task 2` implementation.

**Task 2 Colab URL:** https://colab.research.google.com/drive/1v_vbcmsYiUP_RzkHGFYUZqwU4TJavFy5

## Program main assistant

In the following script I provide a simple demo to understand the woking and flow of the assistant program.

- Given user query, it first recognize the intent `(task_no)`.
- A sample demo is given for `task 1`

In [8]:
from TaskRecognition.rec_model import ExtractNER
from task1_main import task1_bot

if __name__ == '__main__':
    ex_feature = ExtractNER()

    # query = "If q is 1, and the wavelength lambda λ is 1, what is the scattering angle θ in degree? What is the d spacing?"
    # query = "If a material exhibits a spacing between crystal lattice planes d of 6, what does this correspond to in terms of the reciprocal/Fourier space scattering vector q?"
    query = "We want to look at this perovskite to understand its structure. We think 5 seconds of exposure should be sufficient. Theta of 0.2 would be good."

    result = ex_feature.predict_ner(query)
    print(result['ner'])

    task_no = ""
    for ner_obj in result['ner']:
        if 'task_no' in ner_obj:
            task_no = ner_obj["task_no"]
            break

    # Run task based on the intent.
    if task_no:
        if task_no == "task_1":
            task1_bot(ex_feature, result['ner'])
            pass
        if task_no == "task_2":
            print("Performing task 2")


[{'sample_name': 'perovskite'}, {'task_no': 'task_1'}]
Processing sample info. sample_name: perovskite, exposure_time: , angle 


What should be the sample exposure time and incident angle?:  5 seconds exposure time and angle should be 0.5 degree


Running experiments with sample_name: perovskite, exposure_time: 5, and incident angle of 0.5


As you can see from the above output, the model is not very good at extracting all the informations from a large query. This is due to the lack of training data samples.

- Adding more training samples would results in more better accuracy and performance.